# Everactive ENV+ EvalKit Exploration Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/everactive/python-notebook-sample/blob/main/notebook/Everactive_ENVplus_EvalKit_Exploration.ipynb)

This notebook is designed to give you a quickstart to accessing your [Everactive Evironmental+ Evaluation Kit](https://everactive.com/product/environmental-evaluation-kit/) sensor data via the Everactive Data Services API and creating visualizations using your data.

## Before You Begin
* This notebook only supports data from ENV+ EvalKits. Machine Vibration Evaluation Kit data is *not* currently supported.

* This notebook assumes that you are familiar with programming in Python, notebook operation and execution (e.g. JupyterLab or Google Colab), and the `pandas` library.

* We'll use the `altair` library for visualizations. Prior experience with `altair` is not required, but if you would like to learn more about the library and how it works, please visit [their excellent documentation](https://altair-viz.github.io/getting_started/overview.html).

* This notebook pulls *your* sensor data, which will be different from the data used to develop this notebook. Though we've endeavored to craft this notebook to accommodate wide range of returned sensor readings, it is possible that your data might contain an unforseen edge case. If that's the case, and your data breaks the notebook or visualizations, please reach out by [creating a GitHub issue in the notebook repo](https://github.com/everactive/python-notebook-sample/issues).

## Google Colab Setup

If you're running this notebook in Google Colab, there's a few setup steps that we need to run first to prepare the Colab environment. (It may take a few minutes for Colab to install the necessary libraries).

In [ ]:
# Check if we're running locally, or in Google Colab.
try:
    import google.colab
    print("Running in Google Colab, installing everactive_envplus library...")
    !pip install "git+https://github.com/everactive/python-notebook-sample.git"
    print("Install complete.")

except ModuleNotFoundError:
    pass

## Notebook Imports

In [ ]:
import datetime
import getpass
import os
import warnings

import altair as alt
import pandas as pd

import everactive_envplus as ee

# Altair is currently generating FutureWarnings on .iteritems() usage.
warnings.simplefilter(action="ignore", category=FutureWarning)

The `everactive_envplus` library that we will use within this notebook provides a wrapper around the Everactive Data Services API. Its interface is designed to make pulling your Eversensor and Evergateway data easy. 

If you'd like to dive deeper into the Everactive Data Services API interface, please check out our [Data Services API Documentation](https://docs.api.everactive.com/reference/data-services-api-overview).

## Source API Credentials

If you have not already provided your Everactive API credentials as `EVERACTIVE_CLIENT_ID` and `EVERACTIVE_CLIENT_SECRET` environment variables, you will be prompted to enter your Everactive API client id and client secret below:

In [ ]:
CLIENT_ID, CLIENT_SECRET = None, None

In [ ]:
if (os.getenv("EVERACTIVE_CLIENT_ID") is None) or (os.getenv("EVERACTIVE_CLIENT_ID") == "''"):
    CLIENT_ID = getpass.getpass("Enter your Everactive API client id: ")

if os.getenv("EVERACTIVE_CLIENT_SECRET") is None or (os.getenv("EVERACTIVE_CLIENT_SECRET") == "''"):
    CLIENT_SECRET = getpass.getpass("Enter your Everactive API client secret: ")

## Establish a Connection to the API

First, we'll use your API credentials to connect to the Everactive Data Services API.

The `EveractiveApi` object that we create provides methods to fetch Eversensor and Evergateway data from the Everactive Data Services API.

In [ ]:
conn = ee.connection.ApiConnection(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

api = ee.EveractiveApi(api_connection=conn)

## Pull Eversensors Associated with Your Account

To get all of the Eversensors associated with your account, use the `get_all_eversensors()` call. All methods that return API data accept an `output_format` argument that specifies `json` (the default) or `pandas`.

In [ ]:
eversensors = api.get_all_eversensors()
print(f"Fetched {len(eversensors)} eversensors.")

if len(eversensors) == 0:
    raise Exception(
        "No Eversensors were found for your account credentials. "
        "Have you succesfully set up your EvalKit Eversensors?"
    )

In [ ]:
eversensors[0]

In [ ]:
df_eversensors = api.get_all_eversensors(output_format="pandas")
df_eversensors.head()

## Pull Readings for a Single Eversensor

Next, we'll pull readings for an individual Eversensor. The code below selects the first Eversensor returned from the prior call, but feel free to update the code to select a different Eversensor. The `sensor_mac_address` displayed indicates the Eversensor mac address that will be used to pull readings in subsequent API calls.

In [ ]:
sensor_mac_address = eversensors[0]["macAddress"]
sensor_mac_address

### Examine Returned Eversensor Reading Schema

To fetch the most recent reading for a given Eversensor, use the `get_eversensor_last_reading()` method, which takes an argument of the Eversensor mac address.

The returned result contains a collection of data, including value readings from the Eversensor's environmental sensors (temperature, barometric pressure, humidity) and metadata on the operation of the Eversensor itself.

In [ ]:
last_reading = api.get_eversensor_last_reading(sensor_mac_address)
last_reading

### Pull Last 24 Hours of Reported Readings for Eversensor

The API allows you to pull up to 24 hours of readings per call. We'll pull the last 24 hours of readings for our selected Eversensor, based on that Eversensor's most recent reading timestamp.

The `timestamp` field is a unix time stamp, so we can compute 24 hours ago by subtracting the total number of seconds in 24 hours.

In [ ]:
start_time = last_reading["timestamp"] - 24*60*60 +2
end_time = last_reading["timestamp"] + 2

df_readings = api.get_eversensor_readings(
    sensor_mac_address,
    start_time=start_time,
    end_time=end_time,
    output_format="pandas"
)

if df_readings.empty:
    raise Exception(
        f"No readings were found for Eversensor {sensor_mac_address} during requested time period "
        f"({datetime.datetime.fromtimestamp(start_time)} to {datetime.datetime.fromtimestamp(end_time)}). "
        "Have you succesfully set up your EvalKit Eversensors?"
    )

In [ ]:
print(df_readings.shape)
df_readings.head()

### Identify Eversensor Movement Events

If there are detected movement events in your retrieved Eversensor data, you will see a `movementMeasurement_movement` column in the results DataFrame. Below, we'll pull out any movement events present in the Eversensor readings.

In [ ]:
movement_events = []

if "movementMeasurement_movement" in df_readings.columns:
    movement_detected = df_readings["movementMeasurement_movement"].fillna(False)

    for _, row in df_readings[movement_detected].iterrows():
        movement_events.append({
            "macAddress": row["macAddress"],
            "timestamp" : row["timestamp"],
            "readingDate" : row["readingDate"],
        })

df_movements = pd.DataFrame(movement_events)

In [ ]:
df_movements.head()

## Visualize Eversensor Readings Data

Armed with our Eversensor readings, we can now visualize data from the readings. We'll first instantiate the Everactive data visualization color palette for use in our charts, and define some useful constants.

In [ ]:
color = ee.color.ColorPalette()

CHART_HEIGHT = 300
CHART_WIDTH = 700

FULL_DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

SENSOR_SHORT_NAME = f"{sensor_mac_address[-5:]}"

For each chart, we'll prep the data with `pandas` then create the visualization with `altair`.

Altair charts that use `.interactive(bind_y=False)` and `.interactive()` are able to zoom - use the mouse scroll to zoom in and out. Double clicking on the chart will reset it to its original coordinates.

Since we are going to overlay movement events on our environmental and Eversensor data charts, we'll create the `movements` Altair chart for reuse and layering in the subsequent charts. 

In [ ]:
df_movements["legend_label"] = "Movement Event"

# Create movement chart.
movements_chart = (
    alt.Chart(df_movements).mark_rule().encode(
        alt.X("readingDate:T"),
        alt.Color(
            "legend_label",
            legend=alt.Legend(title=None),
            scale=alt.Scale(domain=["Movement Event"], range=[color.violet()])
        ),
        tooltip=[alt.Tooltip("readingDate:T", title="Movement Event", format=FULL_DATETIME_FORMAT)]
    )
)

if df_movements.empty:
    movements = []
else:
    movements = [movements_chart]

### Environmental Data

We'll first visualize the temperature, humidity, and pressure data from the last 24 hours.

#### Temperature

In [ ]:
temperature_readings = []

# Extract and prep the temperature data.
for _, row in df_readings[["readingDate", "temperatureMeasurements"]].iterrows():
    for temp_reading in row["temperatureMeasurements"]:
        # The BME280 sensor used by the ENV+ Eversensor reports two temperatures, off-chip and
        # on-chip. We use the off-chip temperature to approximate ambient temperature.
        if temp_reading["sensorIndex"] == 0:
            temperature_readings.append(
                {
                    "readingDate" : row["readingDate"],
                    # Temperature is reported in Kelvin; we'll convert to Celsius for visualization.
                    "temperature" : temp_reading["value"] - 273.15,
                }
            )

df_plot = pd.DataFrame(temperature_readings)
df_plot["legend_label"] = "Temperature"
df_plot["display_temp_c"] = df_plot["temperature"].apply(lambda x: f"{round(x, 1)} ºC")
df_plot["display_temp_f"] = df_plot["temperature"].apply(lambda x: f"{round(x*(9/5)+32, 1)} ºF")

# Create the temperature chart.
temperatures = alt.Chart(df_plot).mark_circle().encode(
    alt.X("readingDate:T", axis=alt.Axis(title=None)),
    alt.Y("temperature:Q", axis=alt.Axis(title="Temperature (C)")),
    alt.Color(
        "legend_label",
        legend=alt.Legend(title="Legend"),
        scale=alt.Scale(domain=["Temperature"], range=[color.sky()])
    ),
    tooltip=[
        alt.Tooltip("readingDate:T", title="Reading", format=FULL_DATETIME_FORMAT),
        alt.Tooltip("display_temp_c", title="Celsius"),
        alt.Tooltip("display_temp_f", title="Fahrenheit"),
    ]
)

# Render the layered chart.
(
    alt.layer(temperatures, *movements)
        .resolve_scale(color="independent")
        .configure_title(anchor="start")
        .properties(
            title=f"[{SENSOR_SHORT_NAME}] Temperature (Celsius)",
            height=CHART_HEIGHT, width=CHART_WIDTH
        ).interactive(bind_y=False)
)

#### Humidity

In [ ]:
humidity_readings = []

# Extract and prep the humidity data.
for _, row in df_readings[["readingDate", "humidityMeasurements"]].iterrows():
    for humidity_reading in row["humidityMeasurements"]:
        if humidity_reading["sensorIndex"] == 0:
            humidity_readings.append(
                {
                    "readingDate" : row["readingDate"],
                    "humidity" : humidity_reading["value"],
                }
            )

df_plot = pd.DataFrame(humidity_readings)
df_plot["legend_label"] = "Relative Humidity"
df_plot["display_humidity"] = df_plot["humidity"].apply(lambda x: f"{round(x,1)}%")

# Create the humidity chart.
humidity = alt.Chart(df_plot).mark_circle().encode(
    alt.X("readingDate:T", axis=alt.Axis(title=None)),
    alt.Y("humidity:Q", axis=alt.Axis(title="Relative Humidity (%)")),
    alt.Color(
        "legend_label",
        legend=alt.Legend(title="Legend"),
        scale=alt.Scale(domain=["Relative Humidity"], range=[color.sky()])
    ),
    tooltip=[
        alt.Tooltip("readingDate:T", title="Reading", format=FULL_DATETIME_FORMAT),
        alt.Tooltip("display_humidity", title="Relative Humidity"),
    ]
)

# Render the layered chart.
(
    alt.layer(humidity, *movements)
        .resolve_scale(color="independent")
        .configure_title(anchor="start")
        .properties(
            title=f"[{SENSOR_SHORT_NAME}] Relative Humidity",
            height=CHART_HEIGHT, width=CHART_WIDTH
        ).interactive(bind_y=False)
)

#### Barometric Pressure

In [ ]:
# Prep the pressure data.
df_plot = df_readings[["readingDate", "pressureMeasurement"]].copy().reset_index(drop=True)
df_plot["legend_label"] = "Barometric Pressure"
df_plot["display_pressure"] = df_plot["pressureMeasurement"].apply(lambda x: f"{round(x)} hPa")

# Create the pressure chart.
pressure = alt.Chart(df_plot).mark_circle().encode(
    alt.X("readingDate:T", axis=alt.Axis(title=None)),
    alt.Y("pressureMeasurement:Q", axis=alt.Axis(title="Barometric Pressure (hPa)")),
    alt.Color(
        "legend_label",
        legend=alt.Legend(title="Legend"),
        scale=alt.Scale(domain=["Barometric Pressure"], range=[color.sky()])
    ),
    tooltip=[
        alt.Tooltip("readingDate:T", title="Reading", format=FULL_DATETIME_FORMAT),
        alt.Tooltip("display_pressure", title="Barometric Pressure"),
    ]
)

# Render the layered chart.
(
    alt.layer(pressure, *movements)
        .resolve_scale(color="independent")
        .configure_title(anchor="start")
        .properties(
            title=f"[{SENSOR_SHORT_NAME}] Barometric Pressure",
            height=CHART_HEIGHT, width=CHART_WIDTH
        ).interactive(bind_y=False)
)

### Eversensor Data

Visualizing metadata from Eversensor operation can help to give you insight into how your sensor is working. Below, we'll plot RSSI uplink and energy storage data.

#### RSSI Uplink

RSSI stands for "Received Signal Strength Indicator." It is a measurement of how well your Eversensor can hear a signal from the Evergateway. The closer to zero the RSSI Uplink value is, the better the signal.

In [ ]:
# Prep the RSSI data.
df_plot = df_readings[["readingDate", "rssiUplink"]].copy().reset_index(drop=True)
df_plot["legend_label"] = "RSSI Uplink"
df_plot["display_rssi"] = df_plot["rssiUplink"].apply(lambda x: f"{round(x)}")

# Create the RSSI chart.
rssi = alt.Chart(df_plot).mark_circle().encode(
    alt.X("readingDate:T", axis=alt.Axis(title=None)),
    alt.Y("rssiUplink:Q", axis=alt.Axis(title="RSSI Uplink")),    
    alt.Color(
        "legend_label",
        legend=alt.Legend(title="Legend"),
        scale=alt.Scale(domain=["RSSI Uplink"], range=[color.sky()])
    ),
    tooltip=[
        alt.Tooltip("readingDate:T", title="Reading", format=FULL_DATETIME_FORMAT),
        alt.Tooltip("display_rssi", title="RSSI Uplink"),
    ]
)

# Render the layered chart.
(
    alt.layer(rssi, *movements)
        .resolve_scale(color="independent")
        .configure_title(anchor="start")
        .properties(
            title=f"[{SENSOR_SHORT_NAME}] RSSI Uplink",
            height=CHART_HEIGHT, width=CHART_WIDTH
        ).interactive(bind_y=False)
)

#### Stored Energy

To plot the stored energy in our Eversensor, we first need to calculate the energy in the Eversensor's capacitor store. We can do this using the voltage across the Eversensor capacitors, `scap` and `vcap`, along with their capacitance.

*If you'd like to learn more about energy harvesting and management on the Eversensor, check out our [Energy Harvesting Sensors 101 primer](https://everactive-energy-harvesting-sensors-101.streamlit.app/).*

ENV+ Eversensors report a value, the photovoltaic cell (PV IN) rail count (`railCounts_PV_IN_count`), that is proportional to the energy harvested by the sensor. We'll include this value on the chart, so that we can compare the PV IN rail count trend to the trend in energy storage on the Eversensor.

In [ ]:
# Calculate stored energy based on scap and vcap data.
def calculated_eversensor_stored_energy(vcap: float, scap: float) -> float:
    """Calculate and return the Eversensor stored energy in joules."""
    
    envplus_operating_capacitance = 2.5 * 1e-3
    envplus_storage_capacitance = 800 * 1e-3
    
    return (0.5 * envplus_operating_capacitance * (vcap**2)) + (
        0.5 * envplus_storage_capacitance * (scap**2)
    )

df_plot = df_readings[["readingDate", "vcap", "scap"]].copy().reset_index(drop=True)
df_plot["stored_energy"] = df_plot.apply(
    lambda row: calculated_eversensor_stored_energy(row['vcap'], row['scap']), axis=1)

df_plot["legend_label"] = "Stored Energy"
df_plot["display_stored_energy"] = df_plot["stored_energy"].apply(
    lambda x: f"Stored Energy: {round(x,2)} J")

# Create a selection that chooses the nearest point & selects based on x value (readingDate).
nearest = alt.selection(
    type="single",
    nearest=True,
    on="mouseover",
    fields=["readingDate"],
    empty="none"
)

# Create transparent selectors across the chart - this tells us the x value on mouseover.
selectors = alt.Chart(df_plot).mark_point().encode(
    x="readingDate:T", opacity=alt.value(0)
).add_selection(nearest)


# Draw a rule at the location of the selection
rule = alt.Chart(df_plot).mark_rule(color=color.charcoal()).encode(
    x="readingDate:T",
).transform_filter(nearest)

# Create common params for text data labels.
data_label_params = {
    "align" : "left", "dx" : 5, "color" : color.charcoal(), "fontWeight" : 600
}

# Create the stored energy chart.
stored_energy_base = alt.Chart(df_plot).encode(
    alt.X("readingDate:T", axis=alt.Axis(title=None)),
    alt.Y("stored_energy:Q", axis=alt.Axis(title="Stored Energy (J)")),    
)

stored_energy_mark = stored_energy_base.mark_line(point=True).encode(
    alt.Color(
        "legend_label",
        legend=alt.Legend(title="Legend"),
        scale=alt.Scale(domain=["Stored Energy"], range=[color.sky()])
    )
)

stored_energy_area = stored_energy_base.mark_area(opacity=0.1).encode(
    alt.Y("stored_energy:Q", axis=None),    
    alt.Color(
        "legend_label",
        legend=None,
        scale=alt.Scale(domain=["Stored Energy"], range=[color.sky()])
    )
)

# Create mouseover data label for stored energy.
stored_energy_data_label = stored_energy_base.mark_text(**data_label_params, dy=-10).encode(
    alt.Y(axis=None),
    text=alt.condition(nearest, "display_stored_energy", alt.value(' '))
)

# Extract and prep the rail count data.
df_rail_counts = df_readings[["readingDate", "railCounts_PV_IN_count"]].copy().reset_index(drop=True)
df_rail_counts["legend_label"] = "PV IN Rail Count"
df_rail_counts["display_rail_count"] = df_rail_counts["railCounts_PV_IN_count"].apply(
    lambda x: f"PV IN Rail Count: {round(x)}")

# Create the rail counts chart.
rail_count_base = alt.Chart(df_rail_counts).encode(
    alt.X("readingDate:T", axis=alt.Axis(title=None)),
    alt.Y("railCounts_PV_IN_count:Q", axis=alt.Axis(title="PV IN Rail Count")),    
)

rail_count_chart = rail_count_base.mark_circle().encode( 
    alt.Color(
        "legend_label",
        legend=alt.Legend(title=None),
        scale=alt.Scale(domain=["PV IN Rail Count"], range=[color.midnight()])
    )
)

# Create mouseover data label for rail counts.
rail_count_data_label = rail_count_base.mark_text(
    **data_label_params, dy=10
).encode(
    alt.Y(axis=None),
    text=alt.condition(nearest, "display_rail_count", alt.value(' '))
)

# Render the layered chart.
(
    alt.layer(
        stored_energy_mark, stored_energy_area, rail_count_chart, *movements,
        selectors, rule,
        stored_energy_data_label, rail_count_data_label
    )
    .resolve_scale(y="independent", color="independent")
    .configure_title(anchor="start")
    .properties(
            title=f"[{SENSOR_SHORT_NAME}] Stored Energy vs. PV IN Rail Count",
            height=CHART_HEIGHT, width=CHART_WIDTH
        ).interactive(bind_y=False)
)

## Join the Everactive Community

If you enjoyed accessing and exploring your ENV+ EvalKit data, we encourage you to engage further and join our Everactive Community! You can find us on the following platforms:
<p style="text-align: center;">
<a href="https://www.hackster.io/everactive"><img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/hackster.png?raw=true" width="128"/></a>
<img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/spacer.png?raw=true" width="32"/>
<a href="https://www.youtube.com/@EveractiveInc"><img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/youtube.png?raw=true" width="128"/></a>
<img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/spacer.png?raw=true" width="32"/>
<a href="https://everactive.github.io/"><img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/github.png?raw=true" width="128"/></a>
<img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/spacer.png?raw=true" width="32"/>
<a href="https://everactive.com/join-slack"><img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/slack.png?raw=true" width="128"/></a>
<img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/spacer.png?raw=true" width="32"/>
<a href="https://support.everactive.com/hc/en-us"><img src="https://github.com/everactive/python-notebook-sample/blob/main/docs/images/community_logos/everactive.png?raw=true" width="128"/></a></p>